In [ ]:
import torch

from data import data_helper
from data.data_helper import available_datasets
from models import model_factory
from optimizer.optimizer_helper import get_optim_and_scheduler
from torch.nn import functional as F
from torch import nn

from utils.Logger import Logger

import torchvision
import matplotlib.pyplot as plt

from train_jigsaw import Trainer
from utils import vis

class Container():
    pass

args = Container()
args.batch_size = 128
args.n_classes = 7
args.learning_rate = 0.001
args.epochs = 30
args.network = "caffenet"
args.jig_weight = 0.2
args.val_size = 0.1
args.tf_logger = True
args.folder_name = "data_bias"
args.jigsaw_n_classes = 31

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.bias_whole_image in [0.5]: # [None, 0.01, 0.05, 0.1, 0.3, 0.5]:
    for k, x in enumerate(source):
        args.source = source[:k]+source[k+1:]
        args.target = x
        for i in range(3):
            print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                              args.target, 
                                                              args.jigsaw_n_classes,
                                                              i))
            trainer = Trainer(args, device)
            trainer.do_training()


cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 7150, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/data_bias/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.2_178_bias0.5
New epoch - lr: 0.0, 0.001
0/55 of epoch 1/30 jigsaw : 3.587723, class : 1.937965 - acc jigsaw : 0.007812, class : 0.218750 [bs:128]
30/55 of epoch 1/30 jigsaw : 2.105837, class : 0.906803 - acc jigsaw : 0.484375, class : 0.648438 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.803279
Accuracies on test: jigsaw : 1.000000, class : 0.577637
New epoch - lr: 0.0, 0.001
0/55 of epoch 2/30 jigsaw : 2.259349, class : 0.868736 - acc jigsaw : 0.429688, class : 0.679688 [bs:128]
30/55 of epoch 2/30 jigsaw : 1.860950, class : 0.675585 - acc jigsaw : 0.507812, class : 0.757812 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.861286
Accuracies on test: jigsaw : 1.000

30/55 of epoch 23/30 jigsaw : 1.405792, class : 0.434345 - acc jigsaw : 0.632812, class : 0.843750 [bs:128]
Accuracies on val: jigsaw : 0.994956, class : 0.909206
Accuracies on test: jigsaw : 0.991699, class : 0.627441
New epoch - lr: 0.0, 0.001
0/55 of epoch 24/30 jigsaw : 1.436935, class : 0.480378 - acc jigsaw : 0.617188, class : 0.773438 [bs:128]
30/55 of epoch 24/30 jigsaw : 1.319490, class : 0.329928 - acc jigsaw : 0.640625, class : 0.851562 [bs:128]
Accuracies on val: jigsaw : 0.996217, class : 0.918033
Accuracies on test: jigsaw : 0.991699, class : 0.637695
New epoch - lr: 0.0, 0.0001
0/55 of epoch 25/30 jigsaw : 1.182131, class : 0.418881 - acc jigsaw : 0.679688, class : 0.859375 [bs:128]
30/55 of epoch 25/30 jigsaw : 1.295061, class : 0.410009 - acc jigsaw : 0.695312, class : 0.835938 [bs:128]
Accuracies on val: jigsaw : 0.996217, class : 0.914250
Accuracies on test: jigsaw : 0.991699, class : 0.634277
New epoch - lr: 0.0, 0.0001
0/55 of epoch 26/30 jigsaw : 1.351145, class :

Accuracies on val: jigsaw : 1.000000, class : 0.911728
Accuracies on test: jigsaw : 0.993652, class : 0.626953
New epoch - lr: 0.0, 0.001
0/55 of epoch 16/30 jigsaw : 1.229778, class : 0.369303 - acc jigsaw : 0.656250, class : 0.851562 [bs:128]
30/55 of epoch 16/30 jigsaw : 1.679017, class : 0.616807 - acc jigsaw : 0.554688, class : 0.757812 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.924338
Accuracies on test: jigsaw : 0.993164, class : 0.621582
New epoch - lr: 0.0, 0.001
0/55 of epoch 17/30 jigsaw : 1.613308, class : 0.625900 - acc jigsaw : 0.554688, class : 0.710938 [bs:128]
30/55 of epoch 17/30 jigsaw : 1.464645, class : 0.347229 - acc jigsaw : 0.578125, class : 0.882812 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.918033
Accuracies on test: jigsaw : 0.993652, class : 0.628906
New epoch - lr: 0.0, 0.001
0/55 of epoch 18/30 jigsaw : 1.381979, class : 0.507215 - acc jigsaw : 0.640625, class : 0.796875 [bs:128]
30/55 of epoch 18/30 jigsaw : 1.529430, class : 0

Accuracies on test: jigsaw : 0.996094, class : 0.627930
New epoch - lr: 0.0, 0.001
0/55 of epoch 8/30 jigsaw : 1.636025, class : 0.432373 - acc jigsaw : 0.570312, class : 0.835938 [bs:128]
30/55 of epoch 8/30 jigsaw : 1.860127, class : 0.648261 - acc jigsaw : 0.492188, class : 0.773438 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.899117
Accuracies on test: jigsaw : 0.994141, class : 0.611328
New epoch - lr: 0.0, 0.001
0/55 of epoch 9/30 jigsaw : 1.699999, class : 0.392305 - acc jigsaw : 0.523438, class : 0.851562 [bs:128]
30/55 of epoch 9/30 jigsaw : 1.509902, class : 0.577374 - acc jigsaw : 0.593750, class : 0.781250 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.901639
Accuracies on test: jigsaw : 0.995117, class : 0.613281
New epoch - lr: 0.0, 0.001
0/55 of epoch 10/30 jigsaw : 1.582107, class : 0.500576 - acc jigsaw : 0.570312, class : 0.781250 [bs:128]
30/55 of epoch 10/30 jigsaw : 1.468892, class : 0.496568 - acc jigsaw : 0.609375, class : 0.789062 [bs:128]


Dataset size: train 6884, val 763, test 2344
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/data_bias/art_painting-photo-sketch_to_cartoon/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.2_482_bias0.5
New epoch - lr: 0.0, 0.001
0/53 of epoch 1/30 jigsaw : 3.502413, class : 1.995062 - acc jigsaw : 0.023438, class : 0.148438 [bs:128]
30/53 of epoch 1/30 jigsaw : 2.067835, class : 0.991316 - acc jigsaw : 0.492188, class : 0.625000 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.808650
Accuracies on test: jigsaw : 1.000000, class : 0.597696
New epoch - lr: 0.0, 0.001
0/53 of epoch 2/30 jigsaw : 1.797579, class : 0.736055 - acc jigsaw : 0.546875, class : 0.718750 [bs:128]
30/53 of epoch 2/30 jigsaw : 1.818489, class : 0.653567 - acc jigsaw : 0.546875, class : 0.742188 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.828309
Accuracies on test: jigsaw : 0.999573, class : 0.578925
New epoch - lr: 0.0, 0.001
0/53 of epoch 3/30 jigsaw : 1.817294, 

...

Accuracies on val: jigsaw : 0.998795, class : 0.884337
Accuracies on test: jigsaw : 0.992216, class : 0.904192
New epoch - lr: 0.0, 0.001
0/58 of epoch 24/30 jigsaw : 1.238571, class : 0.317565 - acc jigsaw : 0.640625, class : 0.898438 [bs:128]
30/58 of epoch 24/30 jigsaw : 1.335262, class : 0.684674 - acc jigsaw : 0.671875, class : 0.750000 [bs:128]
Accuracies on val: jigsaw : 0.998795, class : 0.886747
Accuracies on test: jigsaw : 0.989820, class : 0.904192
New epoch - lr: 0.0, 0.0001
0/58 of epoch 25/30 jigsaw : 1.257587, class : 0.391364 - acc jigsaw : 0.656250, class : 0.851562 [bs:128]
30/58 of epoch 25/30 jigsaw : 1.438202, class : 0.414617 - acc jigsaw : 0.617188, class : 0.812500 [bs:128]
Accuracies on val: jigsaw : 0.998795, class : 0.881928
Accuracies on test: jigsaw : 0.990419, class : 0.906587
New epoch - lr: 0.0, 0.0001
0/58 of epoch 26/30 jigsaw : 1.335260, class : 0.463961 - acc jigsaw : 0.648438, class : 0.796875 [bs:128]
30/58 of epoch 26/30 jigsaw : 1.376628, class :

Accuracies on test: jigsaw : 0.996407, class : 0.896407
New epoch - lr: 0.0, 0.001
0/58 of epoch 16/30 jigsaw : 1.473256, class : 0.525294 - acc jigsaw : 0.617188, class : 0.843750 [bs:128]
30/58 of epoch 16/30 jigsaw : 1.271965, class : 0.484719 - acc jigsaw : 0.656250, class : 0.789062 [bs:128]
Accuracies on val: jigsaw : 0.996386, class : 0.872289
Accuracies on test: jigsaw : 0.994012, class : 0.897605
New epoch - lr: 0.0, 0.001
0/58 of epoch 17/30 jigsaw : 1.340288, class : 0.448398 - acc jigsaw : 0.656250, class : 0.828125 [bs:128]
30/58 of epoch 17/30 jigsaw : 1.354772, class : 0.401967 - acc jigsaw : 0.664062, class : 0.828125 [bs:128]
Accuracies on val: jigsaw : 0.996386, class : 0.869880
Accuracies on test: jigsaw : 0.994611, class : 0.891617
New epoch - lr: 0.0, 0.001
0/58 of epoch 18/30 jigsaw : 1.557451, class : 0.418175 - acc jigsaw : 0.585938, class : 0.843750 [bs:128]
30/58 of epoch 18/30 jigsaw : 1.417403, class : 0.677124 - acc jigsaw : 0.625000, class : 0.789062 [bs:1

0/58 of epoch 8/30 jigsaw : 1.674801, class : 0.505563 - acc jigsaw : 0.531250, class : 0.789062 [bs:128]
30/58 of epoch 8/30 jigsaw : 1.853751, class : 0.507419 - acc jigsaw : 0.492188, class : 0.820312 [bs:128]
Accuracies on val: jigsaw : 0.998795, class : 0.875904
Accuracies on test: jigsaw : 0.997006, class : 0.895210
New epoch - lr: 0.0, 0.001
0/58 of epoch 9/30 jigsaw : 1.746652, class : 0.609880 - acc jigsaw : 0.515625, class : 0.765625 [bs:128]
30/58 of epoch 9/30 jigsaw : 1.869713, class : 0.605116 - acc jigsaw : 0.531250, class : 0.726562 [bs:128]
Accuracies on val: jigsaw : 0.998795, class : 0.872289
Accuracies on test: jigsaw : 0.997006, class : 0.897006
New epoch - lr: 0.0, 0.001
0/58 of epoch 10/30 jigsaw : 1.631969, class : 0.615171 - acc jigsaw : 0.585938, class : 0.765625 [bs:128]
30/58 of epoch 10/30 jigsaw : 1.697900, class : 0.543232 - acc jigsaw : 0.515625, class : 0.796875 [bs:128]
Accuracies on val: jigsaw : 0.998795, class : 0.857831
Accuracies on test: jigsaw :

Dataset size: train 5457, val 605, test 3929
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/data_bias/art_painting-cartoon-photo_to_sketch/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.2_717_bias0.5
New epoch - lr: 0.0, 0.001
0/42 of epoch 1/30 jigsaw : 3.448815, class : 1.964141 - acc jigsaw : 0.023438, class : 0.164062 [bs:128]
30/42 of epoch 1/30 jigsaw : 2.186985, class : 0.759733 - acc jigsaw : 0.476562, class : 0.718750 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.857851
Accuracies on test: jigsaw : 1.000000, class : 0.441079
New epoch - lr: 0.0, 0.001
0/42 of epoch 2/30 jigsaw : 1.897592, class : 0.683810 - acc jigsaw : 0.515625, class : 0.765625 [bs:128]
30/42 of epoch 2/30 jigsaw : 1.910198, class : 0.740370 - acc jigsaw : 0.500000, class : 0.750000 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.866116
Accuracies on test: jigsaw : 0.999745, class : 0.472130
New epoch - lr: 0.0, 0.001
0/42 of epoch 3/30 jigsaw : 1.835914, 

Accuracies on val: jigsaw : 0.996694, class : 0.907438
Accuracies on test: jigsaw : 0.995928, class : 0.487401
New epoch - lr: 0.0, 0.001
0/42 of epoch 24/30 jigsaw : 1.676131, class : 0.359554 - acc jigsaw : 0.531250, class : 0.851562 [bs:128]
30/42 of epoch 24/30 jigsaw : 1.488432, class : 0.280457 - acc jigsaw : 0.593750, class : 0.898438 [bs:128]
Accuracies on val: jigsaw : 0.996694, class : 0.909091
Accuracies on test: jigsaw : 0.996437, class : 0.494019
New epoch - lr: 0.0, 0.0001
0/42 of epoch 25/30 jigsaw : 1.460041, class : 0.263753 - acc jigsaw : 0.593750, class : 0.914062 [bs:128]
30/42 of epoch 25/30 jigsaw : 1.449136, class : 0.271480 - acc jigsaw : 0.632812, class : 0.867188 [bs:128]
Accuracies on val: jigsaw : 0.996694, class : 0.910744
Accuracies on test: jigsaw : 0.996437, class : 0.482311
New epoch - lr: 0.0, 0.0001
0/42 of epoch 26/30 jigsaw : 1.562381, class : 0.360410 - acc jigsaw : 0.539062, class : 0.859375 [bs:128]
30/42 of epoch 26/30 jigsaw : 1.521350, class :

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % ("-".join(args.source),
                                                            args.target,args.epochs, args.jig_weight))

In [ ]:
dataloaders = {"train": data_helper.get_dataloader(args.source, args.jigsaw_n_classes, "train"),
           "val": data_helper.get_dataloader(args.target, args.jigsaw_n_classes, "val")}
dataset_sizes = {"train": len(dataloaders["train"].dataset),
                "val": len(dataloaders["val"].dataset)}
print(dataset_sizes)

In [ ]:
def get_optim_and_scheduler(network, epochs, lr):
    from torch import optim
    optimizer = optim.SGD(network.get_params(lr), weight_decay=.0005, momentum=.9, nesterov=True, lr=lr)
    step_size = int(epochs * .8)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)
    print(step_size)
    return optimizer, scheduler

model_ft = model_factory.get_network("caffenet")(jigsaw_classes=args.jigsaw_n_classes+1, classes=args.n_classes)
model_ft = model_ft.to(device)
# print(model_ft)

optimizer, scheduler = get_optim_and_scheduler(model_ft, args.epochs, args.learning_rate)

In [ ]:
def do_epoch(model, source, target, optimizer, logger, device):
    criterion = nn.CrossEntropyLoss()
    model.train()
    for it, ((data, jig_l, class_l), d_idx) in enumerate(source):
        data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
        
        optimizer.zero_grad()
        
        jigsaw_logit, class_logit = model(data)
        jigsaw_loss = criterion(jigsaw_logit, jig_l)
        class_loss = criterion(class_logit[d_idx!=target_id], class_l[d_idx!=target_id])
        _, cls_pred = class_logit.max(dim=1)
        _, jig_pred = jigsaw_logit.max(dim=1)
        loss = class_loss + jigsaw_loss * jig_weight
        
        loss.backward()
        optimizer.step()

        logger.log(it, len(source), {"jigsaw": jigsaw_loss.item(), "class": class_loss.item()},
                  {"jigsaw": torch.sum(jig_pred == jig_l.data).item(), "class":torch.sum(cls_pred == class_l.data).item()},
                  data.shape[0])
        del loss, class_loss, jigsaw_loss, jigsaw_logit, class_logit

    model.eval()
    with torch.no_grad():
        jigsaw_correct = 0
        class_correct = 0
        total = 0
        for it, ((data, jig_l, class_l), d_idx) in enumerate(target):
            data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
            jigsaw_logit, class_logit = model(data)
            _, cls_pred = class_logit.max(dim=1)
            _, jig_pred = jigsaw_logit.max(dim=1)
            class_correct += torch.sum(cls_pred == class_l.data)
            jigsaw_correct += torch.sum(jig_pred == jig_l.data)
            total += data.shape[0]
        logger.log_test({"jigsaw": float(jigsaw_correct) / total,
                         "class": float(class_correct) / total})


def do_training(args, model, source, target, optimizer, scheduler, device):
    logger = Logger(args)
    for k in range(args.epochs):
        scheduler.step()
        logger.new_epoch(scheduler.get_lr())
        do_epoch(model, source, target, optimizer, logger, device)
    return logger, model

In [ ]:
jig_weight = args.jig_weight
logger, model = do_training(args, model_ft, dataloaders["train"], dataloaders["val"], optimizer, scheduler, device)

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(args.source),args.target,args.epochs, jig_weight))

In [ ]:
def to_plt(inp):
    import numpy as np
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = model_ft.features[0] # models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(source),target,epochs, jig_weight))

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.show()

In [ ]:
for e,k in enumerate(range(0,l,int(updates))):
    print(k, logger.val_acc[e])

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
from os.path import join, dirname
# from data.JigsawLoader import JigsawTestDataset
import torch
import matplotlib.pyplot as plt
import numpy as np


def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
    return inp

# dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)
# test = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
iter_c = iter(data_helper.get_dataloader("photo", 31, "train"))
(tmp, v, c), d = next(iter_c)
for x in range(5):
    image = tmp[x]
    image = torchvision.utils.make_grid(tmp[x],1,normalize=True)
    plt.imshow(to_plt(image))
    plt.show()
    print(v[x],c[x])
    
print(v.max(), v.min())

In [ ]:
v

In [ ]:
[d[d==k].shape for k in [0,1]]


In [ ]:
from data.JigsawLoader import JigsawDataset
from PIL import Image
import torchvision.transforms as transforms

class JigsawTestDataset(JigsawDataset):
    def __init__(self, *args, **xargs):
        super().__init__(*args, **xargs)
        self._augment_tile = transforms.Compose([
#             transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]
        img = Image.open(framename).convert('RGB')
        img = self._image_transformer(img)

        w = float(img.size[0]) / self.grid_size
        n_grids = self.grid_size ** 2
        tiles = [None] * n_grids
        for n in range(n_grids):
            y = int(n / self.grid_size)
            x = n % self.grid_size
            tile = img.crop([x * w, y * w, (x + 1) * w, (y + 1) * w])
            tile = self._augment_tile(tile)
            tiles[n] = tile

        data = torch.stack(tiles, 0)
        return self.returnFunc(data), 0, int(self.labels[index])


In [ ]:
dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)

In [ ]:
import torch
import random

In [ ]:
true = 0
for x in range(100):
    true += 0.1 > random.random()
true